In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from sentence_transformers import SentenceTransformer, InputExample, models, losses, util
from torch import nn
from torch.utils.data import DataLoader

from pathlib import Path

# Fix the random seed for reproducibility
# !! Important !! : do not change this
seed = 1234
np.random.seed(seed)


DATA_PATH = Path('data/original-dataset')
VOCAB_PATH = DATA_PATH.joinpath('imdb.vocab')
TRAIN_BOW_PATH = DATA_PATH.joinpath('train/labeledBow.feat')
TEST_BOW_PATH = DATA_PATH.joinpath('test/labeledBow.feat')
TRAIN_FILES = DATA_PATH.glob('train/[pn][oe][sg]/*.txt')
TEST_FILES = DATA_PATH.glob('test/[pn][oe][sg]/*.txt')
COLORMAP = 'seismic'


# data processing constants
MAX_DF = 0.4
MIN_DF = 2
PCA_COMPONENTS = 100

C:\Users\User\miniconda3\envs\cs178\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = {
    'pos': [],
    'neg': []
}

for path, i in zip(TRAIN_FILES, range(25001)):
    with open(path, 'r', encoding='utf8') as file:
        rating = int(path.stem.split('_')[1])
        contents = file.read().strip()
        
        key = 'pos' if rating > 5 else 'neg'
        train_data[key].append((contents, rating))

print(f'parsed {sum(len(d) for d in train_data.values())} reviews')
print(f'positive: {len(train_data["pos"])}')
print(f'negative: {len(train_data["neg"])}')

# test_data = []

# for path, i in zip(TEST_FILES, range(25001)):
#     with open(path, 'r', encoding='utf8') as file:
#         rating = int(path.stem.split('_')[1])
#         contents = file.read().strip()
        
#         test_data.append((contents, rating))
        
# print(f'parsed {len(test_data)} test reviews')

parsed 25000 reviews
positive: 12500
negative: 12500


In [25]:
# generate train example tuples ([text, text], similarity)
# anchor is a review
# positive is review that should be similar to anchor
# negative is a review that should not be similar to anchor

def similarity(r1, r2):
    """
    Returns the similarity between two texts given thier rating
    """
    if r1 <= 5 and r2 <= 5:
        return 1.0
    elif r1 > 5 and r2 > 5:
        return 1.0
    else:
        return 0.0
    
# number of train examples
n = 100000
print(f'{n} train examples')

merged_data = train_data['pos'] + train_data['neg']

pair_idxs = np.random.randint(0, len(merged_data), (n, 2))
print(pair_idxs[0])

train_examples = [
    InputExample(
        texts=[merged_data[i1][0], merged_data[i2][0]],
        label=similarity(merged_data[i1][1], merged_data[i2][1])
    ) 
    for i1, i2 in pair_idxs
]

print(train_examples[0].texts[0])
print('-----')
print(train_examples[0].texts[1])
print('-----')
print(train_examples[0].label)

100000 train examples
[19627 10336]
I was looking forward to this ride, and was horribly disappointed.<br /><br />And I am very easily amused at roller coaster and amusement park rides.<br /><br />The roller coaster part was just okay - and that was all of about 30 seconds of a 90 second ride. <br /><br />It was visually dull and poorly executed. <br /><br />It was trying desperately to be like a mixture of the far superior Indiana Jones and Space Mountain rides and Disneyland, and failed in every aspect.<br /><br />It was not thrilling or exciting in the least.
-----
This is of of Sammo's great early comedy films. This isn't a parody of enter the dragon, the main character (Sammo) is obsessed with Bruce Lee and emulates him freakishly well for a man of his size. Nominal story about how his fighting keeps causing his loved ones trouble - then fighting. Oh, the fighting. Good, fast-paced scenes with high impact (the white guy who plays a boxer looks like he really gets hurt by one of Sa

In [4]:
# build custom sBert model
model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
dense_model = models.Dense(in_features=384, out_features=100, activation_function=nn.ReLU())
model.add_module('dense', dense_model)

# define dataload and loss function used to train model
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

# # torch.cuda.empty_cache()
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

Iteration:   2%|█▍                                                                | 131/6250 [01:59<1:33:25,  1.09it/s]


Iteration:   4%|██▊                                                               | 263/6250 [04:01<1:31:32,  1.09it/s]


Iteration:   6%|████▏                                                             | 395/6250 [06:02<1:29:25,  1.09it/s]


Iteration:   8%|█████▌                                                            | 527/6250 [08:03<1:27:33,  1.09it/s]


Iteration:  11%|██████▉                                                           | 659/6250 [10:04<1:25:43,  1.09it/s]


...

Iteration:  93%|████████████████████████████████████████████████████████████▍    | 5807/6250 [1:28:53<06:44,  1.10it/s]


Iteration:  95%|█████████████████████████████████████████████████████████████▊   | 5939/6250 [1:30:54<04:45,  1.09it/s]


Iteration:  97%|███████████████████████████████████████████████████████████████▏ | 6071/6250 [1:32:55<02:44,  1.09it/s]


Iteration:  99%|████████████████████████████████████████████████████████████████▌| 6203/6250 [1:34:56<00:43,  1.09it/s]


Epoch: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [1:35:40<00:00, 5740.14s/it]


In [27]:
model = SentenceTransformer('sBERT_IMBD_Embedding_Model')

data = merged_data
r1 = data[np.random.randint(0, len(data))]
r2 = data[np.random.randint(0, len(data))]

print(r1[0])
print(r1[1])
print('-----')
print(r2[0])
print(r2[1])
print()

encodings = model.encode([text for text, rating in data], normalize_embeddings=True, show_progress_bar=True)
print(util.cos_sim(encodings, encodings))

print(encodings[0])


basic_model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
encodings = basic_model.encode([r1[0], r2[0]])
print(util.cos_sim(encodings, encodings))

Although not the best Anime I have ever seen but I enjoyed Lady Death.<br /><br />I have never read the comic book and just saw this at the video store and decided to give it a chance.<br /><br />The animation was OK, I got the sense of the 80's anime from it which is what set it off for me. Why everyone else hates that is beyond me.<br /><br />Character development was fine. I like how they brought the transition from Hope to Lady Death around. for you who don't like it, obviously wasn't paying attention. Lucifer tells you how it happens, and she used his words.<br /><br />Creamtor was a nice mentor/soldier for her. his dark bruiting style was perfect for this kind of movie.<br /><br />I think everyone here who has bashed this needs to take another look at it and reconsider. cause everything people have bashed this can be said about everyones favorite anime Vampire Hunter D
8
-----
I must admit, out of the EROS MOVIE COLLECTION, this has to be the one that I love the most as well as o

Batches: 100%|███████████████████████████████████████████████████████████████████████| 782/782 [03:43<00:00,  3.50it/s]


tensor([[1.0000, 0.9457, 0.9744,  ..., 0.0504, 0.0359, 0.0272],
        [0.9457, 1.0000, 0.9480,  ..., 0.0671, 0.0452, 0.0323],
        [0.9744, 0.9480, 1.0000,  ..., 0.0480, 0.0315, 0.0267],
        ...,
        [0.0504, 0.0671, 0.0480,  ..., 1.0000, 0.9716, 0.9681],
        [0.0359, 0.0452, 0.0315,  ..., 0.9716, 1.0000, 0.9835],
        [0.0272, 0.0323, 0.0267,  ..., 0.9681, 0.9835, 1.0000]])
[0.29171646 0.         0.         0.01768552 0.12590608 0.
 0.         0.0595879  0.         0.17915688 0.         0.00160482
 0.         0.01703718 0.         0.         0.         0.04743262
 0.03839002 0.         0.04364038 0.         0.         0.
 0.         0.3771691  0.         0.02885771 0.         0.
 0.         0.         0.         0.16622043 0.         0.
 0.         0.         0.         0.0919247  0.         0.
 0.         0.         0.         0.25631836 0.22993614 0.
 0.         0.         0.02403845 0.         0.         0.
 0.05380134 0.         0.         0.         0.32148594

In [63]:
model.save('sBERT_IMBD_Embedding_Model')

In [38]:
our_model = SentenceTransformer('sBERT_IMBD_Embedding_Model')
basic_model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')

model = basic_model

pos_data = np.array([text for text, rating in train_data['pos']])
neg_data = np.array([text for text, rating in train_data['neg']])

n = 100
pos_sample = pos_data[np.random.randint(0, len(pos_data), (n,2))]
neg_sample = neg_data[np.random.randint(0, len(neg_data), (n,2))]

avg = 0
for pair in pos_sample:
    embeddings = model.encode(pair, normalize_embeddings=True)
    cos_sim = np.dot(embeddings[0], embeddings[1])
    avg += cos_sim
print(f'Average of Cosine Similarity between positive reviews:\n{float(avg)/n}')

avg = 0
for pair in neg_sample:
    embeddings = model.encode(pair, normalize_embeddings=True)
    cos_sim = np.dot(embeddings[0], embeddings[1])
    avg += cos_sim
print(f'Average of Cosine Similarity between negative reviews:\n{float(avg)/n}')

avg = 0
for pair in zip(pos_sample[:,0], neg_sample[:,0]):
    embeddings = model.encode(pair, normalize_embeddings=True)
    cos_sim = np.dot(embeddings[0], embeddings[1])
    avg += cos_sim
print(f'Average of Cosine Similarity between different polarity reviews:\n{float(avg)/n}')



# pos_embeddings = our_model.encode(pos_sample, normalize_embeddings=True, show_progress_bar=True)
# neg_embeddings = our_model.encode(neg_sample, normalize_embeddings=True, show_progress_bar=True)

# print(pos_embeddings.shape)
# print('positive embedding similarity')
# pos_sim = util.cos_sim(pos_embeddings, pos_embeddings)
# print(pos_sim)
# print(pos_sim.shape)
# print()


Average of Cosine Similarity between positive reviews:
0.2943987012282014
Average of Cosine Similarity between negative reviews:
0.3267371230572462
Average of Cosine Similarity between different polarity reviews:
0.30120386272668837
